## 读取查看数据

In [1]:
import jieba
import jieba.analyse
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series
from collections import Counter
from pyecharts import options as opts
from pyecharts.charts import Bar,Geo,WordCloud

In [2]:
# 最多显示10行
pd.set_option('display.max_rows', 10)

In [3]:
df = pd.read_csv('data/raw_data.csv')

In [4]:
data = df.copy()

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3721 entries, 0 to 3720
Data columns (total 3 columns):
城市    3721 non-null object
线路    3721 non-null object
站名    3721 non-null object
dtypes: object(3)
memory usage: 87.3+ KB


In [6]:
data.head(5)

,城市,线路,站名
0,北京,S1线,金安桥
1,北京,S1线,四道桥
2,北京,S1线,桥户营
3,北京,S1线,上岸
4,北京,S1线,栗园庄


## 评估和清洗数据

In [7]:
# 查看有无重复的数据
data[data.duplicated()].head(5)
#data[data.loc[: , '站名'] == '东川路']

,城市,线路,站名
516,上海,地铁5号线,东川路
678,上海,地铁10号线,新江湾城
679,上海,地铁10号线,殷高东路
680,上海,地铁10号线,三门路
681,上海,地铁10号线,江湾体育场


In [8]:
# 清除重复的数据并重建索引
data.drop_duplicates(keep='first',inplace=True)
data.reset_index(drop=True,inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3594 entries, 0 to 3593
Data columns (total 3 columns):
城市    3594 non-null object
线路    3594 non-null object
站名    3594 non-null object
dtypes: object(3)
memory usage: 84.3+ KB


In [9]:
data.tail()

,城市,线路,站名
3589,石家庄,轨道交通3号线,新百广场
3590,石家庄,轨道交通3号线,东里
3591,石家庄,轨道交通3号线,槐安桥
3592,石家庄,轨道交通3号线,西三教
3593,石家庄,轨道交通3号线,石家庄站


In [10]:
# 查看站名信息
data['线路'].unique()

array(['S1线', '地铁1号线', '地铁2号线', '地铁4号线大兴线', '地铁5号线', '地铁6号线', '地铁7号线',
       '地铁8号线', '地铁8号线南段', '地铁9号线', '地铁10号线', '地铁13号线', '地铁14号线东段',
       '地铁14号线西段', '地铁15号线', '地铁16号线', '地铁八通线', '地铁昌平线', '地铁房山线', '机场线',
       '西郊线', '地铁燕房线', '地铁亦庄线', '地铁2号线东延线', '地铁3号线', '地铁4号线', '地铁11号线',
       '地铁12号线', '地铁17号线', '磁悬浮', '轨道交通浦江线', '地铁3号线(北延段)', '地铁14号线',
       '地铁14号线支线(知识城线)', '地铁21号线', 'APM线', '广佛线', '地铁1号线(罗宝线)',
       '地铁2号线(蛇口线)', '地铁3号线(龙岗线)', '地铁4号线(龙华线)', '地铁5号线(环中线)', '轨道交通1号线',
       '轨道交通2号线', '轨道交通3号线', '轨道交通4号线', '轨道交通6号线', '轨道交通7号线', '轨道交通8号线',
       '轨道交通11号线', '轨道交通21号线(阳逻线)', '津滨轻轨9号线', '地铁S1号线(机场线)',
       '地铁S3号线(宁和线)', '地铁S7号线(宁溧线)', '地铁S8号线(宁天线)', '地铁S9号线(宁高线)', '迪士尼綫',
       '港島綫', '南港島綫', '西鐵綫', '荃灣綫', '將軍澳綫', '東涌綫', '東鐵綫', '機場快綫', '觀塘綫',
       '馬鞍山綫', '轨道交通3号线北延伸段(空港线)', '轨道交通5号线', '轨道交通6号线支线(国博线)',
       '轨道交通10号线', '轨道交通环线', '地铁保税区线', '地铁九里线', '地铁九里支线', '轨道交通4号线支线',
       '城郊线', '磁浮快线'], dtype=object)

In [11]:
# 去掉站名前的 地铁，轨道交通等字样
data['线路'] = data['线路'].str.strip('地铁')
data['线路'] = data['线路'].str.strip('轨道交通')

In [12]:
data['线路'].unique()

array(['S1线', '1号线', '2号线', '4号线大兴线', '5号线', '6号线', '7号线', '8号线', '8号线南段',
       '9号线', '10号线', '13号线', '14号线东段', '14号线西段', '15号线', '16号线', '八通线',
       '昌平线', '房山线', '机场线', '西郊线', '燕房线', '亦庄线', '2号线东延线', '3号线', '4号线',
       '11号线', '12号线', '17号线', '磁悬浮', '浦江线', '3号线(北延段)', '14号线',
       '14号线支线(知识城线)', '21号线', 'APM线', '广佛线', '1号线(罗宝线)', '2号线(蛇口线)',
       '3号线(龙岗线)', '4号线(龙华线)', '5号线(环中线)', '21号线(阳逻线)', '津滨轻轨9号线',
       'S1号线(机场线)', 'S3号线(宁和线)', 'S7号线(宁溧线)', 'S8号线(宁天线)', 'S9号线(宁高线)',
       '迪士尼綫', '港島綫', '南港島綫', '西鐵綫', '荃灣綫', '將軍澳綫', '東涌綫', '東鐵綫', '機場快綫',
       '觀塘綫', '馬鞍山綫', '3号线北延伸段(空港线)', '6号线支线(国博线)', '环线', '保税区线', '九里线',
       '九里支线', '4号线支线', '城郊线', '磁浮快线'], dtype=object)

## 探索性分析和可视化

### 拥有地铁城市的个数，地铁线路，站点总数

In [13]:
len(data['城市'].unique())

32

In [14]:
data_lines = data.groupby(['城市','线路']).count().reset_index()
data_lines

,城市,线路,站名
0,上海,10号线,31
1,上海,11号线,38
2,上海,12号线,32
3,上海,13号线,31
4,上海,16号线,13
...,...,...,...
177,香港,荃灣綫,16
178,香港,西鐵綫,12
179,香港,觀塘綫,17
180,香港,迪士尼綫,2


In [15]:
df_stations = data.groupby(['城市','站名'],as_index=False).count()
df_stations.rename(columns={'线路':'数量'},inplace=True)
df_stations

,城市,站名,数量
0,上海,七宝,1
1,上海,七莘路,1
2,上海,三林,1
3,上海,三林东,1
4,上海,三门路,1
...,...,...,...
3179,香港,馬鞍山,1
3180,香港,鰂魚涌,2
3181,香港,黃埔,1
3182,香港,黃大仙,1


#### 我们看到共有32个城市，182条地铁线路，3183个地铁站点。

### 各城市拥有的线路，站点情况一览

In [16]:
# 查看各个城市的地铁线路数
line_data = data_lines.groupby(['城市'])['线路'].count().sort_values(ascending=False)
line_data

城市
北京    23
上海    18
广州    16
香港    11
重庆    10
      ..
无锡     2
厦门     1
贵阳     1
佛山     1
东莞     1
Name: 线路, Length: 32, dtype: int64

In [17]:
geo = Geo()

geo.add_schema(maptype="china")
geo.add("geo", [list(z) for z in zip(line_data.index.tolist(), line_data.values.tolist())])
geo.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
geo.set_global_opts(
    visualmap_opts=opts.VisualMapOpts(max_=25,is_piecewise=True),
    title_opts=opts.TitleOpts(title="已开通地铁城市分布情况"))

geo.render_notebook()

In [18]:
b = Bar()
b.add_xaxis(line_data.index.tolist())
b.add_yaxis("地铁线路", line_data.values.tolist())
b.set_global_opts(title_opts=opts.TitleOpts(title="各城市地铁线路情况"),
                  yaxis_opts=opts.AxisOpts(name="数量"),
                  xaxis_opts=opts.AxisOpts(name="城市"))
b.render_notebook()

In [19]:
bins = range(0,30,5)
bins = pd.cut(line_data,bins)
line_gpdata = bins.groupby(bins.values).count()
line_gpdata.index = [str(ind.left) + '~' + str(ind.right)+ '条' for ind in line_gpdata.index]

In [20]:
c = Bar()
c.add_xaxis(line_gpdata.index.tolist())
c.add_yaxis("数据", line_gpdata.values.tolist())
c.set_global_opts(title_opts=opts.TitleOpts(title="城市地铁线路数量分布情况"),
                  yaxis_opts=opts.AxisOpts(name="城市个数"),
                  xaxis_opts=opts.AxisOpts(name="地铁线路数量"))
c.render_notebook()

In [21]:
# 统计每个城市的地铁站总数
stations_info = df_stations.groupby('城市')['数量'].count().sort_values(ascending=False)
stations_info

城市
上海     345
北京     329
广州     225
武汉     189
深圳     166
      ... 
哈尔滨     25
佛山      25
厦门      24
贵阳      24
东莞      15
Name: 数量, Length: 32, dtype: int64

In [22]:
d = Bar()
d.add_xaxis(stations_info.index.tolist())
d.add_yaxis("数据", stations_info.values.tolist())
d.set_global_opts(title_opts=opts.TitleOpts(title="各城市地铁站数量"),
                  yaxis_opts=opts.AxisOpts(name="地铁站数量"),
                  xaxis_opts=opts.AxisOpts(name="城市"))
d.render_notebook()

In [23]:
# 哪个城市哪条线路地铁站数最多
data_lines.sort_values(by='站名',ascending=False)

,城市,线路,站名
20,北京,10号线,45
151,重庆,3号线,39
1,上海,11号线,38
74,天津,6号线,38
114,武汉,2号线,38
...,...,...,...
106,昆明,6号线,4
40,北京,机场线,4
166,长沙,磁浮快线,3
17,上海,磁悬浮,2


### 查看地铁站名的高频名字

In [24]:
text = ''
station_info = df_stations['站名'].values.tolist()
for stats in station_info:
    text += ' '.join(jieba.cut(stats,cut_all=True))
    text += ' '

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/3b/7ygpd2wj01g67xqzgplmf52m0000gn/T/jieba.cache
Loading model cost 1.116 seconds.
Prefix dict has been built succesfully.


In [25]:
tags = jieba.analyse.extract_tags(text, topK=50, withWeight=True)

In [26]:
label = []
attr = []
for item in tags:
    label.append(item[0])
    attr.append(int(item[1]*1000))

In [27]:
w = WordCloud()
w.add("", zip(label, attr), word_size_range=[20, 100], shape='star')
w.set_global_opts(title_opts=opts.TitleOpts(title="地铁站名词云图"))
w.render_notebook()

In [28]:
c = Counter(text)

In [29]:
ser = Series(c)
cha_data = ser.sort_values(ascending=False)[1:11]
cha_data

路    705
大    391
南    231
东    211
园    205
中    201
山    191
北    181
西    181
站    160
dtype: int64

In [30]:
e = Bar()
e.add_xaxis(cha_data.index.tolist())
e.add_yaxis("名字", cha_data.values.tolist())
e.set_global_opts(title_opts=opts.TitleOpts(title="地铁站高频名字图"),
                  yaxis_opts=opts.AxisOpts(name="数量"),
                  xaxis_opts=opts.AxisOpts(name="名字"))
e.render_notebook()

### 城市地铁站名特点

In [31]:
df_sh = df_stations[df_stations['城市']=='上海']
df_sh[df_sh['站名'].str.contains('路')]

,城市,站名,数量
1,上海,七莘路,1
4,上海,三门路,1
5,上海,三鲁公路,1
6,上海,上南路,1
7,上海,上大路,1
...,...,...,...
339,上海,龙华中路,2
341,上海,龙溪路,1
342,上海,龙漕路,2
343,上海,龙耀路,1


In [32]:
df_stations.groupby(['城市'])['数量'].count()

城市
上海    345
东莞     15
佛山     25
北京    329
南京    159
     ... 
重庆    162
长春     84
长沙     65
青岛     79
香港     92
Name: 数量, Length: 32, dtype: int64

#### 我们看到在上海345个地铁站里，有210个都包含'路'字

In [33]:
df_men = df_stations[df_stations['站名'].str.contains('门')].groupby(['城市'])['数量'].count().sort_values(ascending=False)

In [34]:
f = Bar()
f.add_xaxis(df_men.index.tolist())
f.add_yaxis("个数", df_men.values.tolist())
f.set_global_opts(title_opts=opts.TitleOpts(title="地铁站名较爱用'门'的城市"),
                  yaxis_opts=opts.AxisOpts(name="地铁站个数"),
                  xaxis_opts=opts.AxisOpts(name="城市"))
f.render_notebook()

In [35]:
df_bj = df_stations[df_stations['城市']=='北京']
df_bj[df_bj['站名'].str.contains('门')]

,城市,站名,数量
400,北京,东直门,3
424,北京,健德门,1
425,北京,光熙门,1
440,北京,前门,1
445,北京,北京大学东门,1
...,...,...,...
667,北京,西红门,1
671,北京,角门东,1
672,北京,角门西,2
690,北京,阜成门,1


In [36]:
df_nj = df_stations[df_stations['城市']=='南京']
df_nj[df_nj['站名'].str.contains('门')]

,城市,站名,数量
715,南京,上元门,1
721,南京,中华门,1
733,南京,仙鹤门,1
750,南京,卡子门,1
772,南京,安德门,2
...,...,...,...
816,南京,玄武门,1
840,南京,草场门·南艺·二师,1
844,南京,西安门,1
857,南京,集庆门大街,1


In [37]:
df_xa = data[data['城市'] == '西安']
df_xa[df_xa['站名'].str.contains('门')]

,城市,线路,站名
2825,西安,1号线,开远门
2827,西安,1号线,玉祥门
2831,西安,1号线,朝阳门
2833,西安,1号线,通化门
2847,西安,2号线,南稍门
...,...,...,...
2862,西安,3号线,延平门
2870,西安,3号线,延兴门
2873,西安,3号线,通化门
2877,西安,3号线,广泰门


#### 我们看到地铁站名较爱用'门'的城市前三分别是北京，南京，西安，都是有着深厚历史底蕴的城市